In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from textblob import TextBlob
import emoji
from IPython.display import display

In [2]:
import pprint # For "pretty-printing" of Python objects -- easier to interprate output 
pp = pprint.PrettyPrinter() # Create a pprint object

Import preprocess.py file which contains defined functions that will load objects and create dataframes.
We should add to this file as we unpack these columns from the json

In [4]:
from preprocess import tweet_open, insta_comment, extract_emojis

### Check out the whole json file

In [5]:
# We load in the json output from the IG sraper as a df

IG_media = pd.read_json('iamalicaldwell.json')
display(IG_media.head())
pp.pprint(list(IG_media.columns))

,__typename,comments,comments_disabled,dimensions,display_url,edge_media_preview_like,edge_media_to_caption,edge_media_to_comment,gating_info,id,...,media_preview,owner,shortcode,tags,taken_at_timestamp,thumbnail_resources,thumbnail_src,urls,username,video_view_count
0,GraphVideo,"{'data': [{'created_at': 1536899339, 'id': '17...",False,"{'height': 607, 'width': 1080}",https://scontent-lga3-1.cdninstagram.com/vp/8b...,{'count': 2202},{'edges': [{'node': {'text': 'Meeting Aretha F...,{'count': 77},NaN,1867926020993499648,...,ACoXLHUGgBDgtn1J4x9c1Ncag0n+ryuPQ1hvdBzlgc/lV6...,{'id': '359244331'},BnsNNHEBsGY,"[ComingSoon, Subscribe, ArethaFranklin, AliCal...",1536897261,"[{'config_height': 150, 'config_width': 150, '...",https://scontent-lga3-1.cdninstagram.com/vp/91...,[https://scontent-lga3-1.cdninstagram.com/vp/d...,iamalicaldwell,0
1,GraphVideo,"{'data': [{'created_at': 1536876260, 'id': '17...",False,"{'height': 423, 'width': 750}",https://scontent-lga3-1.cdninstagram.com/vp/c9...,{'count': 2568},{'edges': [{'node': {'text': 'Never in my life...,{'count': 138},NaN,1867600018689977344,...,ACoXxkVZGwQSOBxTSgRmUHIHek8wdRwadF87Evk+tIv8yQ...,{'id': '359244331'},BnrDFJwh2vN,"[ComingSoon, AliCaldwell, StoneCold, DemiLovat...",1536856372,"[{'config_height': 150, 'config_width': 150, '...",https://scontent-lga3-1.cdninstagram.com/vp/31...,[https://scontent-lga3-1.cdninstagram.com/vp/6...,iamalicaldwell,0


['__typename',
 'comments',
 'comments_disabled',
 'dimensions',
 'display_url',
 'edge_media_preview_like',
 'edge_media_to_caption',
 'edge_media_to_comment',
 'gating_info',
 'id',
 'is_video',
 'media_preview',
 'owner',
 'shortcode',
 'tags',
 'taken_at_timestamp',
 'thumbnail_resources',
 'thumbnail_src',
 'urls',
 'username',
 'video_view_count']


- Each row is a unique posting to user's IG profile.
- There are many useful columns, **most** of which are dictionaries

# Check out comments
Since each row is a unique post made user, we'll focus on the first one which is a video (see link to it)
- https://www.instagram.com/p/BnsNNHEBsGY/?utm_source=ig_web_button_share_sheet
- FYI, she has a new post from today, 9/15, so we may want to scrape that

In [17]:
comments = insta_comment('iamalicaldwell.json')
print('There are', comments.shape[0], 'comments.')
comments.head()

There are 76 comments.


,created_at,id,owner,text
0,1536899339,17978534839016109,"{'id': '144873547', 'profile_pic_url': 'https:...",Yes queen ❤️
1,1536899816,17964347710106185,"{'id': '1031014256', 'profile_pic_url': 'https...",She'd be proud you sing her music well.
2,1536899866,17888244430260360,"{'id': '194960641', 'profile_pic_url': 'https:...",Chills!!!!
3,1536900043,17950907089144935,"{'id': '3032556430', 'profile_pic_url': 'https...",Queeeeennnn
4,1536900050,17881014838264700,"{'id': '3032556430', 'profile_pic_url': 'https...",Singggggg😍😍😍😍😍


- The created_at column is in a date format specific to instagram...research how to handle that
- Looking at this post now, there are 94 comments...scrape the rest
- Probably should drop the owner column and replace with just the id since the user name is not important
- Also need to write into the preprocess.py file a function that will extract all these comments and append them to the IG_media dataframe

# Getting user id's

In [21]:
all_users = comments['owner']
all_users.head()

0    {'id': '144873547', 'profile_pic_url': 'https:...
1    {'id': '1031014256', 'profile_pic_url': 'https...
2    {'id': '194960641', 'profile_pic_url': 'https:...
3    {'id': '3032556430', 'profile_pic_url': 'https...
4    {'id': '3032556430', 'profile_pic_url': 'https...
Name: owner, dtype: object

Another dictionary that will need to be handled in preprocess.py